In [ ]:
%matplotlib inline
import os
import yt
yt.mylog.setLevel("WARNING")
import numpy as np
import matplotlib
matplotlib.rcParams['font.family'] = 'stixgeneral'
matplotlib.rcParams['figure.dpi'] = 150
import matplotlib.pyplot as plt

from yt_conduction_fields import *

In [ ]:
ds = yt.load('/home/ychen/Mount/gdrive/2015_production_runs/0529_L45_M10_b1_h1/data/MHD_Jet_hdf5_plt_cnt_0000')
sp1 = ds.sphere([0,0,0], (180, 'kpc'))
sp2 = ds.sphere([0,0,0], (0.5, 'kpc'))
sp = sp1 - sp2

ds.current_time.in_units('Myr')

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'jet ')
plot.zoom(4)
plot.set_zlim('jet ', 1E-7, 1)
plot.annotate_contour('jet ', ncont=2, clim=(1E-4, 1E-3), take_log=True)
plot.show()


In [ ]:
plot = yt.SlicePlot(ds, 'y', 'xray_emissivity_0.1_100_keV')
plot.zoom(4)
plot.show()


In [ ]:
plot = yt.SlicePlot(ds, 'y', 'xray_cooling_time', )
plot.set_unit('xray_cooling_time', 'Gyr')
plot.zoom(4)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'temperature')
plot.set_unit('temperature', 'keV', equivalency='thermal')
plot.zoom(4)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'entropy', data_source=sp)
plot.set_zlim('entropy', 30, 250)
plot.zoom(4)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'H_nuclei_density')
plot.zoom(4)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'temperature_gradient_magnitude', data_source=sp, width=(240, 'kpc'))
plot.set_unit('temperature_gradient_magnitude', 'K/pc')
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'temperature_gradient_x', data_source=sp, width=(240, 'kpc'))
plot.set_unit('temperature_gradient_x', 'K/pc')
plot.set_cmap('temperature_gradient_x', 'seismic')
plot.set_zlim('temperature_gradient_x', -1E3, 1E3)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'spitzer_conduction_coefficient', data_source=sp, width=(240, 'kpc'))
plot.set_zlim('spitzer_conduction_coefficient', 1E29, 1E32)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'spitzer_heat_flux_x', data_source=sp, width=(240, 'kpc'))
plot.set_log('spitzer_heat_flux_x' , True, linthresh=1E-3)
plot.set_cmap('spitzer_heat_flux_x', 'seismic')
plot.set_zlim('spitzer_heat_flux_x', -1E-1, 1E-1)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'spitzer_heat_flux_divergence', data_source=sp, width=(240, 'kpc'))

plot.set_log('spitzer_heat_flux_divergence' , True, linthresh=1E-25)
plot.set_cmap('spitzer_heat_flux_divergence', 'seismic_r')
plot.set_zlim('spitzer_heat_flux_divergence', -1E-23, 1E-23)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'spitzer_heating_rate', data_source=sp, width=(240, 'kpc'))

plot.set_log('spitzer_heating_rate' , True, linthresh=1E40)
plot.set_cmap('spitzer_heating_rate', 'seismic')
plot.set_zlim('spitzer_heating_rate', -1E42, 1E42)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'total_heating_rate', data_source=sp, width=(240, 'kpc'))

plot.set_log('total_heating_rate' , True, linthresh=1E40)
plot.set_cmap('total_heating_rate', 'seismic')
plot.set_zlim('total_heating_rate', -1E42, 1E42)
plot.show()

In [ ]:
plot = yt.SlicePlot(ds, 'y', 'total_cooling_time', data_source=sp, width=(240, 'kpc'))

plot.set_log('total_cooling_time' , False)
plot.set_cmap('total_cooling_time', 'seismic')
plot.set_zlim('total_cooling_time', -1E3, 1E3)
plot.show()

In [ ]:
extrema = {'entropy': (20, 250)}
logs = {'entropy': True}
fields = ['spitzer_heating_rate', 'xray_luminosity_0.1_100_keV', 'cell_mass']
prof_entropy = yt.create_profile(sp, 'entropy', fields, weight_field=None, 
                         extrema=extrema, logs=logs)

In [ ]:
fig = plt.figure(figsize=(6,4))
plt.step(prof_entropy.x, -prof_entropy['xray_luminosity_0.1_100_keV'],
         label='X-ray Cooling', linestyle='dotted')
plt.step(prof_entropy.x, prof_entropy['spitzer_heating_rate'],
         label='Spitzer Thermal Conduction', linestyle='dashed')
plt.step(prof_entropy.x, prof_entropy['spitzer_heating_rate']-prof_entropy['xray_luminosity_0.1_100_keV'], 
         label='Total $f_{\mathrm{Sp}}$ = 1', linewidth=1)
plt.step(prof_entropy.x, 0.1*prof_entropy['spitzer_heating_rate']-prof_entropy['xray_luminosity_0.1_100_keV'], 
         label=r'Total $f_{\mathrm{Sp}}$ = 0.1', linewidth=2)
plt.ylim(-1E45, 1E45)
plt.yscale('symlog', linthreshy=1E42)
yticks = [-1E45, -1E44, -1E43, -1E42, 0, 1E42, 1E43, 1E44, 1E45]

plt.yticks(yticks, yticks)
plt.ylabel('Heating(+)/Cooling(-) rate (erg/s)')
plt.semilogx()
plt.xlim(20, 250)
entropy_ticks = [20, 30, 40, 60, 80, 100, 150, 200]
plt.xticks(entropy_ticks, entropy_ticks)
plt.xlabel(r'Entropy (cm$^2\ $ keV)')
plt.axhline(0, ls='-', lw=1, color='k')
plt.text(170, 3E44, '%.0f Myr' % ds.current_time.in_units('Myr'))
plt.legend(loc=3, fontsize=9, ncol=2, columnspacing=0.5)

In [ ]:
extrema = {'entropy': (20, 250), 'spherical_radius': ((10, 'kpc'), (150, 'kpc'))}
fields = ['spitzer_heating_rate', 'xray_luminosity_0.1_100_keV', 'cell_mass']
prof = yt.create_profile(sp, ['entropy', 'spherical_radius'], fields, weight_field=None,
                         extrema=extrema)

In [ ]:
pp = yt.PhasePlot.from_profile(prof, figure_size=6)
pp.set_unit('spherical_radius', 'kpc')
pp.set_cmap('spitzer_heating_rate', 'RdBu_r')
pp.set_log('spitzer_heating_rate', False)
pp.set_zlim('spitzer_heating_rate', -1E44, 1E44)
pp.set_ylim(20, 150)
pp.set_xlim(20, 250)

plot2 = pp['spitzer_heating_rate']

r_ticks = [30, 40, 50, 60, 80, 100, 150]

plot2.axes.set_yticks(r_ticks)
#plot2.axes.yaxis.set_ticklabels(r_ticks)
plot2.axes.set_yticklabels(r_ticks)
plot2.axes.set_yticklabels([], minor=True)
#print(plot2.axes.yaxis.get_ticklabels())


entropy_ticks = [20, 30, 40, 60, 80, 100, 150, 200]
plot2.axes.set_xticks(entropy_ticks)
plot2.axes.set_xticklabels(entropy_ticks)

plot2

In [ ]:
extrema = {'spherical_radius': ((10, 'kpc'), (150, 'kpc'))}
logs = {'spherical_radius': True}
fields = ['spitzer_heating_rate', 'xray_luminosity_0.1_100_keV', 'cell_mass']
prof_radius = yt.create_profile(sp, 'spherical_radius', fields, weight_field=None, 
                         extrema=extrema, logs=logs)

In [ ]:
plt.clf()
plt.step(prof_radius.x.in_units('kpc'), -prof_radius['xray_luminosity_0.1_100_keV'],
         label='X-ray Cooling', linestyle='dotted')
plt.step(prof_radius.x.in_units('kpc'), prof_radius['spitzer_heating_rate'],
         label='Spitzer Thermal Conduction', linestyle='dashed')
plt.step(prof_radius.x.in_units('kpc'), prof_radius['spitzer_heating_rate']-prof_radius['xray_luminosity_0.1_100_keV'], 
         label='Total $f_{\mathrm{Sp}}$ = 1', linewidth=1)
plt.step(prof_radius.x.in_units('kpc'), 0.1*prof_radius['spitzer_heating_rate']-prof_radius['xray_luminosity_0.1_100_keV'], 
         label=r'Total $f_{\mathrm{Sp}}$ = 0.1', linewidth=2)
plt.ylim(-1E45, 1E45)
plt.yscale('symlog', linthreshy=1E42)
yticks = [-1E45, -1E44, -1E43, -1E42, 0, 1E42, 1E43, 1E44, 1E45]

plt.yticks(yticks, yticks)
plt.ylabel('Heating(+)/Cooling(-) rate (erg/s)')
plt.semilogx()
plt.xlim(20, 150)
radius_ticks = [20, 30, 40, 60, 80, 100, 150]
plt.xticks(radius_ticks, radius_ticks)
plt.xlabel(r'Radius (kpc)')
plt.axhline(0, ls='-', lw=1, color='k')
plt.text(110, 3E44, '%.0f Myr' % ds.current_time.in_units('Myr'))
plt.legend(loc=3, fontsize=9, ncol=2, columnspacing=0.5)

In [ ]:
Lmin,Lmax = 1E40, 1E44
line = np.logspace(np.log10(Lmin), np.log10(Lmax))
plt.scatter(prof_radius['xray_luminosity_0.1_100_keV'], prof_radius['spitzer_heating_rate'],
            c=prof_radius.x.in_units('kpc'))
plt.colorbar()
plt.plot(line, line)
plt.plot(line, line*0.1)

plt.xlim(Lmin, Lmax)
plt.ylim(Lmin, Lmax)
plt.semilogx()
plt.semilogy()